In [ ]:
import os
import sys

# This helps to load custom script modules from scripts folder
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

from data_processing import load_data

zip_file_path = ''
